In [27]:
import pandas as pd
import os
import datetime
import numpy as np


def recursive_file_gen(my_root_dir):
    for root, dirs, files in os.walk(my_root_dir):
        for file in files:
            yield os.path.join(root, file)
   

In [28]:
Weeks_2018_Q2=[datetime.date(2018,5,12)+datetime.timedelta(days=x*7) for x in range(13)]

Q2_2018_weekly_data=list(recursive_file_gen("/home/jian/BigLots/2018_by_weeks/"))
Q2_2018_weekly_data=[x for x in Q2_2018_weekly_data if ("SalesWeekly" in x)]

Q2_2018_weekly_data_list=[]
for week in Weeks_2018_Q2:
    weekly_file=[x for x in Q2_2018_weekly_data if str(week) in x]
    if len(weekly_file)==1:
        Q2_2018_weekly_data_list=Q2_2018_weekly_data_list+weekly_file


In [36]:
df_output=pd.DataFrame()
df_by_store=pd.DataFrame()
for file in Q2_2018_weekly_data_list:
    try:
        df=pd.read_table(file,dtype=str,sep="|",usecols=['location_id','week_end_dt','subclass_gross_sales_amt'])
        df['subclass_gross_sales_amt']=df['subclass_gross_sales_amt'].astype(float)
        df=df.rename(columns={"subclass_gross_sales_amt":"sales"})
        df=df.groupby(['location_id','week_end_dt'])['sales'].sum().to_frame().reset_index()
        df['type']=np.where(df['location_id']=="6990","Ecommerce","InStore")
        df_by_store=df_by_store.append(df)
        df=df.groupby(['type','week_end_dt'])['sales'].sum().to_frame().reset_index()
        df_output=df_output.append(df)
    except:
        df=pd.read_table(file,dtype=str,sep="|",usecols=['location_id','week_end_dt','class_gross_sales_amt'])
        df['class_gross_sales_amt']=df['class_gross_sales_amt'].astype(float)
        df=df.rename(columns={"class_gross_sales_amt":"sales"})
        df=df.groupby(['location_id','week_end_dt'])['sales'].sum().to_frame().reset_index()
        df['type']=np.where(df['location_id']=="6990","Ecommerce","InStore")
        df_by_store=df_by_store.append(df)
        df=df.groupby(['type','week_end_dt'])['sales'].sum().to_frame().reset_index()
        df_output=df_output.append(df)

In [38]:
type_df=df_output.groupby(['type'])['sales'].sum().to_frame().reset_index()

In [39]:
writer=pd.ExcelWriter("/home/jian/Projects/Big_Lots/Analysis/2019_Q1/Planner_Request/Q2_2018_Sales_Ecom_InStore_Bruce_0320/BL_2018Q2_total_Sales.xlsx",engine="xlsxwriter")
type_df.to_excel(writer,"Total_Type",index=False)
df_output.to_excel(writer,"Total_Type_week",index=False)
df_by_store.to_excel(writer,"Store_Week",index=False)
writer.save()